In [1]:
import os
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")
%cd .
# %load_ext autoreload
# %autoreload 2

/home/jh/code/til/til-23-cv


In [2]:
%%sh
./setup.sh

Not on competition platform, exiting...


## YOLO Object Detection


In [3]:
from ultralytics import YOLO
from PIL import Image
from time import sleep

In [4]:
# NOTE: Run this cell as its needed for training later.
model = YOLO("yolov5m6u.pt")

### Playtest Model


In [5]:
result = model.predict(
    "http://farm5.staticflickr.com/4032/5134536454_184138d8b1_z.jpg",
    half=True,
    device="cuda:0",
)

In [6]:
def imshow(arr):
    display(Image.fromarray(arr[..., ::-1]))


imshow(result[0].plot())

### Preview Augmentations

This was used heavily to adjust augmentations.

In [9]:
def end_early(_):
    raise KeyboardInterrupt


tempmodel = YOLO("yolov8n.pt")
tempmodel.add_callback("on_train_batch_end", end_early)
try:
    tempmodel.train(
        cfg="cfg/custom.yaml",
        data=os.path.abspath("data/til23plushonly/dataset.yaml"),
        name="temp/temp",
        deterministic=True,
        workers=0,
        batch=9,
        amp=False,
        verbose=False,
        device="cpu",
    )
except KeyboardInterrupt:
    pass
sleep(1)
for im in list(tempmodel.trainer.plots.keys())[1:]:
    display(Image.open(im))


### Training

See `notebooks/data.ipynb` for how to convert `til23plush` dataset to `til23plushonly`.


In [ ]:
model.train(
    cfg="cfg/custom.yaml",
    data=os.path.abspath("data/til23plushonly/dataset.yaml"),
    name="custom/yolov5m6u",
    workers=8,
    batch=8,
)

**NOTE: Above should produce `runs/detect/custom/yolov5m6u/weights/best.pt` which is used as `models/det.pt`!**

### Inference

NOTE: This is for testing Object Detection and isn't the actual inference pipeline.

In [5]:
model_ckpt = "models/det.pt"
im_root = "data/til23plush/images"
split = "val"

In [8]:
model = YOLO(model_ckpt)

In [ ]:
# NOTE: IT WILL APPEND TO EXISTING PREDICTIONS. MOVE CURRENT PREDICTIONS ELSEWHERE FIRST.
model.fuse()
results = model.predict(
    source=os.path.join(im_root, split),
    conf=0.7,
    iou=0.7,
    half=True,
    save_txt=True,
    save_conf=True,
    device="cuda:0",
    imgsz=1280,
    stream=True,
    name=f"predict/{split}",
)
for _ in results:
    pass